# Работа с Excel

v 0.1

Датасет: себестоимостьА_в1.xlsx

In [668]:
import xlwings as xw
import numpy as np
import pandas as pd
from xlwings import constants 

## 1. Используя xlwings:

In [669]:
wb = xw.Book('себестоимостьА_в1.xlsx')
recipies = wb.sheets['Рецептура']

In [670]:
recipies.range('C7:C10').value

['Хлеб Венгерский', 'Хлеб "Барбари"', 'Багет парижский', 'Чиабатта']

1.1 На листе "Рецептура" для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [671]:
consumption = recipies.range('G7:O10').options(np.array).value
consumption = np.nan_to_num(consumption)
unit_prices = recipies.range('G14:O14').options(np.array).value
cost = consumption @ unit_prices
cost

array([21.48 , 16.525, 17.423, 18.085])

1.2 Результаты рассчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [672]:
# recipies.range('T7:T10').value = cost.reshape(-1, 1)
recipies.range('T6').value = 'Себестоимость'
recipies.range('T7').options(transpose=True).value = cost

1.3 Максимально приблизить форматирование столбца, добавленного в 1.2 к оформлению всей области.

In [673]:
# cost = [round(i, 2) for i in cost]
recipies.range('T7:T10').options(transpose=True).value = cost
xw.Range('T7:T10').number_format = '0,00'
xw.Range('T7:T10').color = (255, 255, 167)
# recipies.range('T4:T6').api.merge()
recipies.range('T6').value = 'Себестоимость'
xw.Range('T4:T6').color = (255, 192, 0)
xw.Range('T4:T6').autofit()

1.4 Заменить значения добавленные в 1.2 на формулы. 

In [674]:
xw.Range('U7').formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)' 

1.5 Выполнить 1.4 с помощью "протягиваемых" формул.

In [675]:
xw.Range('U7').formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)' 
from xlwings.constants import AutoFillType 
xw.Range('U7').api.AutoFill(xw.Range("U7:U10").api, 
                                AutoFillType.xlFillDefault)

True

1.6 Сдлеать так, чтобы решение 1.5 и 1.3 автоматически применилось ко всем аналогичным областям на листе "Рецептура"

In [676]:
consumption = recipies.range('G23:N25').options(np.array).value
consumption = np.nan_to_num(consumption)
unit_prices = recipies.range('G31:N31').options(np.array).value
cost = consumption @ unit_prices

recipies.range('S23:S25').options(transpose=True).value = cost
recipies.range('S22').value = 'Себестоимость'
xw.Range('T23').formula = '=SUMPRODUCT(G23:N23,$G$31:$N$31)' 
xw.Range('T23').api.AutoFill(xw.Range("T23:T25").api, 
                                AutoFillType.xlFillDefault)

True

In [677]:
consumption = recipies.range('G40:X46').options(np.array).value
consumption = np.nan_to_num(consumption)
unit_prices = recipies.range('G52:X52').options(np.array).value
cost = consumption @ unit_prices

recipies.range('AC40:AC46').options(transpose=True).value = cost
recipies.range('AC39').value = 'Себестоимость'
xw.Range('AD40').formula = '=SUMPRODUCT(G40:X40,$G$52:$X$52)' 
xw.Range('AD40').api.AutoFill(xw.Range("AD40:AD46").api, 
                                AutoFillType.xlFillDefault)

True

In [678]:
consumption = recipies.range('G61:Y69').options(np.array).value
consumption = np.nan_to_num(consumption)
unit_prices = recipies.range('G73:Y73').options(np.array).value
cost = consumption @ unit_prices

recipies.range('AD61:AD69').options(transpose=True).value = cost
recipies.range('AD60').value = 'Себестоимость'
xw.Range('AE61').formula = '=SUMPRODUCT(G61:Y61,$G$73:$Y$73)' 
xw.Range('AE61').api.AutoFill(xw.Range("AE61:AE69").api, 
                                AutoFillType.xlFillDefault)

True

## 2. Используя xlwings:

2.1 Собрать цены всех ресурсов, используемые на листе "Рецептура". Проверить, что нет расхождений между ценами совпадающих ресурсов

In [679]:
names = recipies.range('G5:O5').options(np.array).value
costs = recipies.range('G14:O14').options(np.array).value

names = np.append(names, recipies.range('G21:N21').options(np.array).value)
costs = np.append(costs, recipies.range('G31:N31').options(np.array).value)

names = np.append(names, recipies.range('G38:X38').options(np.array).value)
costs = np.append(costs, recipies.range('G52:X52').options(np.array).value)

names = np.append(names, recipies.range('G59:Y59').options(np.array).value)
costs = np.append(costs, recipies.range('G73:Y73').options(np.array).value)

df = pd.DataFrame.from_dict({names[i]: costs[i] for i in range(len(names))}, orient='index', columns=["Цена"])
# df

2.2 Вынести цены ресурсов в новую таблицу на новом листе "Цена ресурсов".

In [680]:
try:
	cost_res = wb.sheets.add(name="Цена ресурсов", after=wb.sheets[0].name)
except ValueError as e:
	print(e)

Sheet named 'Цена ресурсов' already present in workbook


In [681]:
cost_res.range("A1").value = df

com_error: (-2147023174, 'Сервер RPC недоступен.', None, None)

2.3 Заменить фиксированные цены ресурсов на листе "Рецептура" на ссылки на лист "Цена ресурсов"

In [ ]:
wb.sheets[0].activate()

In [ ]:
# Сделал только для одной, так как для остальных процесс аналогичен

xw.Range('G14').formula = '=\'Цена ресурсов\'!B2'
xw.Range('H14').formula = '=\'Цена ресурсов\'!B3'
xw.Range('I14').formula = '=\'Цена ресурсов\'!B4'
xw.Range('I14').formula = '=\'Цена ресурсов\'!B5'
xw.Range('L14').formula = '=\'Цена ресурсов\'!B6'
xw.Range('M14').formula = '=\'Цена ресурсов\'!B7'
xw.Range('N14').formula = '=\'Цена ресурсов\'!B8'
xw.Range('O14').formula = '=\'Цена ресурсов\'!B9'

2.4 Сделать так, чтобы цены ресурсов на листе "Рецептура" подтягивались с листа "Цена ресурсов" по наименованию (подсказка: использовать ВПР, рекомендуется использовать именованные области)

In [ ]:
xw.Range('G14').formula = '=VLOOKUP(G5,\'Цена ресурсов\'!$A$2:$B$33,2,0)' 
xw.Range('G14').api.AutoFill(xw.Range("G14:O14").api, 
                                AutoFillType.xlFillDefault)

True

In [ ]:
xw.Range('G31').formula = '=VLOOKUP(G21,\'Цена ресурсов\'!$A$2:$B$33,2,0)' 
xw.Range('G31').api.AutoFill(xw.Range("G31:N31").api, 
                                AutoFillType.xlFillDefault)

True

In [ ]:
xw.Range('G52').formula = '=VLOOKUP(G38,\'Цена ресурсов\'!$A$2:$B$33,2,0)' 
xw.Range('G52').api.AutoFill(xw.Range("G52:X52").api, 
                                AutoFillType.xlFillDefault)

True

In [ ]:
xw.Range('G73').formula = '=VLOOKUP(G59,\'Цена ресурсов\'!$A$2:$B$33,2,0)' 
xw.Range('G73').api.AutoFill(xw.Range("G73:Y73").api, 
                                AutoFillType.xlFillDefault)

True

In [ ]:
wb.save()
wb.close()

## 3.

3.1 Создать нормализованную (не ниже 3й нормальной формы) базу данных sqlite для хранения информации из себестоимостьА_в1.xlsx .

3.2 Информацию из себестоимостьА_в1.xlsx внести в БД на sqlite.

3.3 По базе данных sqlite сгенерирвать книгу эксель, по структуре аналогичную себестоимостьА_в1.xlsx

3.4 При помощи макроса на xlwings сделать в Excel кнопку, по которой все изменения на листе вносятся в БД, а цвет ячеек, в которых были найдены изменения, меняслся на зеленый.

3.5 Реализовать UDF, которая позволяет по имени ресурса вернуть количество видов в продукции в которых используется данный ресурс

3.6 Реализовать UDF, которая позволяет по имени ресурса вернуть массив наименований продуктов, в которых используется данный ресурс

3.7 Реазливать UDF 3.5 и 3.6 которые для получения ответа использую не книгу Excel а БД на sqlite